In [17]:
from numba import njit,prange, get_thread_id, get_num_threads
import numpy as np
from Swarm_IQ_L import decode_and_evaluate, encode_prod_plan, decode_prod_plan
from CLSP_instances import *
import time
from helper_functions import rounded
from Bees import ABC
from tqdm.notebook import tqdm

In [14]:

@njit
def iterate_abc(
    X, Q, O, trial, fitness,
    gbest_X, gbest_Q, gbest_O, gbest_val,
    demand, production_times, setup_times, capacities,
    production_costs, setup_costs, inventory_costs,
    limit, K_onlookers,
    rand_partner_e, rand_phi_e, rand_Xflip_e, rand_Oflip_e,
    rand_partner_o, rand_phi_o, rand_Xflip_o, rand_Oflip_o, 
    best_idx, threshold1, threshold2
):
    SN, M, T = X.shape

    # 1. Employed bees phase
    for i in range(SN):
        k = rand_partner_e[i]
        if k == i:
            continue
            
        phi   = rand_phi_e[i]

        # generate candidate
        Xc = X[i].copy()
        Qc = Q[i].copy()
        Oc = O[i].copy()
        #position update
        for m in range(M):
            for t in range(T):
                
                if rand_Xflip_e[i, m, t] < threshold1 and X[i, m, t] != X[k, m, t] :
                    Xc[m, t] = 1.0 - Xc[m, t]
                    
                if rand_Oflip_e[i, m, t] < threshold1 and  O[i, m, t] != O[k, m, t] :
                    Oc[m, t] = 1.0 - Oc[m, t]
                    
                val = Q[i, m, t] + np.random.uniform(-1,1) * (Q[i, m, t] - Q[k, m, t])
                if val < 0.0:
                    val = 0.0
                elif val > 1.0:
                    val = 1.0
                Qc[m, t] = val
                

        
        fit_c = decode_and_evaluate(
            Xc, Qc, Oc, demand,
            setup_costs, production_costs,
            production_times, setup_times,
            capacities, inventory_costs
        )

        # compare & update
        if (fit_c[0] < fitness[i, 0]) or (fit_c[0] == fitness[i, 0] and fit_c[1] < fitness[i, 1]):
            X[i]          = Xc
            Q[i]          = Qc
            O[i]          = Oc
            fitness[i, 0] = fit_c[0]
            fitness[i, 1] = fit_c[1]
            trial[i]      = 0
        else:
            trial[i] += 1

    # 2. Onlooker bees phase
    w_sum = 0.0
    w     = np.empty(SN, dtype=np.float64)
    for i in range(SN):
        w_sum += 1.0 / (1.0 + fitness[i, 0] )

    for o in range(K_onlookers):
        # roulette‐wheel select source idx
        threshold = abs(rand_phi_o[o]) * w_sum
        accum     = 0.0
        idx       = 0
        for j in range(SN):
            accum += 1.0 / (1.0 +fitness[j, 0])
            if accum >= threshold:
                idx = j
                break

        k = rand_partner_o[o]
        if k == idx:
                continue

        phi   = rand_phi_o[o] 

        # generate candidate
        Xc = X[idx].copy()
        Qc = Q[idx].copy()
        Oc = O[idx].copy()

        # update position
        for m in range(M):
            for t in range(T):
                
                if rand_Xflip_o[o, m, t] < threshold2 and X[idx, m, t] != X[k, m, t] :
                    Xc[m, t] = 1.0 - Xc[m, t]

                if rand_Oflip_o[o, m, t] < threshold2 and O[idx, m, t] != O[k, m, t] :
                    Oc[m, t] = 1.0 - Oc[m, t]

                val = Q[idx, m, t] + phi  * (Q[idx, m, t] - Q[k, m, t])
                if val < 0.0:
                    val = 0.0
                elif val > 1.0:
                    val = 1.0
                Qc[m, t] = val
                

        fit_c = decode_and_evaluate(
                Xc, Qc, Oc, demand,
                setup_costs, production_costs,
                production_times, setup_times,
                capacities, inventory_costs)

        #  compare & update
        if (fit_c[0] < fitness[idx, 0]) or (fit_c[0] == fitness[idx, 0] and fit_c[1] < fitness[idx, 1]):
            
            
            X[idx]          = Xc
            Q[idx]          = Qc
            O[idx]          = Oc
            fitness[idx, 0] = fit_c[0]
            fitness[idx, 1] = fit_c[1]
            trial[idx]      = 0
            
        else:
            trial[idx] += 1

    # 3. Scout bee phase (on‐demand)
    for i in range(SN):
        if i == best_idx:          # protect only the true best slot
                continue
        if trial[i] > limit:
            for m in range(M):
                for t in range(T):
                    O[i, m, t] = 1 if np.random.rand() < 0.5 else 0
                    X[i, m, t] = 1 if np.random.rand() < 0.5 else 0
                    Q[i, m, t] = np.random.rand()

            trial[i] = 0


            fit_i = decode_and_evaluate(
                    X[i], Q[i], O[i], demand,
                    setup_costs, production_costs,
                    production_times, setup_times,
                    capacities, inventory_costs)
    
            fitness[i, 0] = fit_i[0]
            fitness[i, 1] = fit_i[1]

    # 4. Update global bests

    best_local_idx = best_idx
    best_val_0 = gbest_val[0, 0]
    best_val_1 = gbest_val[0, 1]
    
    for i in range(SN):
        fi0, fi1 = fitness[i, 0], fitness[i, 1]
        if (fi0 < best_val_0) or (fi0 == best_val_0 and fi1 < best_val_1):
            best_local_idx = i
            best_val_0 = fi0
            best_val_1 = fi1
    
    if best_local_idx != best_idx:
        gbest_val[0] = fitness[best_local_idx].copy()

        gbest_X[:] = X[best_local_idx].copy()
        gbest_Q[:] = Q[best_local_idx].copy()
        gbest_O[:] = O[best_local_idx].copy()
        best_idx = best_local_idx

    return (
        X, Q, O, trial, fitness,
        gbest_X, gbest_Q, gbest_O, gbest_val, best_idx)

In [15]:


class ABC:
    def __init__(
        self, n_bees, config, limit, K_onlookers, threshold1, threshold2):
        
        # Problem config
        self.cfg = config()
        self.SN = n_bees
        self.M = self.cfg.M
        self.T = self.cfg.T
        self.threshold1 = threshold1
        self.threshold2 = threshold2

        # Scout threshold
        self.limit = limit
        # number of onlooker 
        self.K = K_onlookers 

        # Initialize solution arrays
        self.X = np.random.randint(0, 2, size=(self.SN, self.M, self.T)).astype(np.int8)
        self.O = np.random.randint(0, 2, size=(self.SN, self.M, self.T)).astype(np.int8)
        self.setups  = np.random.randint(0, 2, size=(self.SN, self.M, self.T)).astype(np.int8)
        self.Q = np.random.rand(self.SN, self.M, self.T)

        # Trial counters
        self.trial = np.zeros(self.SN, dtype=np.int32)

        # Evaluate initial fitness
        self.fitness = np.full((self.SN, 2), np.inf)
        for i in range(self.SN):
            
            fit = decode_and_evaluate(
                    self.X[i], self.Q[i], self.O[i], self.cfg.demand,
                    self.cfg.setup_costs, self.cfg.production_costs,
                    self.cfg.production_times, self.cfg.setup_times,
                    self.cfg.capacities, self.cfg.inventory_costs)

            self.fitness[i, 0] = fit[0]
            self.fitness[i, 1] = fit[1]


        # Global best iniitalization
        # find minimal constraint violations
        min_viols = np.min(self.fitness[:, 0])
        # indices with minimal violations
        candidates = np.where(self.fitness[:, 0] == min_viols)[0]
        # among them, pick the one with minimal objective cost
        obj_vals = self.fitness[candidates, 1]
        self.best_idx = candidates[np.argmin(obj_vals)]
        # set global best value, X, and Q
        self.gbest_val = self.fitness[self.best_idx:self.best_idx+1].copy()
        self.gbest_X = self.X[self.best_idx].copy()
        self.gbest_Q = self.Q[self.best_idx].copy()   
        self.gbest_O = self.O[self.best_idx].copy()   
        
        
    def step(self):
        # pre-generate random values
        self.rand_partner_empl = np.random.randint(0,self.SN,self.SN)
        self.rand_phi_empl    = np.random.rand(self.SN)*2-1
        self.rand_Xflip_empl    = np.random.rand(self.SN,self.M,self.T)
        self.rand_Oflip_empl    = np.random.rand(self.SN,self.M,self.T)
        self.rand_partner_onl = np.random.randint(0,self.SN,self.K)
        self.rand_phi_onl    = np.random.rand(self.K)* 2 - 1.0
        self.rand_Xflip_onl    = np.random.rand(self.K,self.M,self.T)
        self.rand_Oflip_onl    = np.random.rand(self.K,self.M,self.T)
    
        (self.X, self.Q, self.O, self.trial, self.fitness,
         self.gbest_X, self.gbest_Q, self.gbest_O, self.gbest_val, 
         self.best_idx) = iterate_abc(
            self.X, self.Q, self.O, self.trial, self.fitness,
            self.gbest_X, self.gbest_Q, self.gbest_O, self.gbest_val,
            self.cfg.demand, self.cfg.production_times, self.cfg.setup_times,
            self.cfg.capacities, self.cfg.production_costs, self.cfg.setup_costs,
            self.cfg.inventory_costs, self.limit, self.K,
            self.rand_partner_empl, self.rand_phi_empl, self.rand_Xflip_empl, self.rand_Oflip_empl,
            self.rand_partner_onl, self.rand_phi_onl, self.rand_Xflip_onl, self.rand_Oflip_onl,
            self.best_idx, self.threshold1, self.threshold2
        )
    
    def optimize_pbar(self, n_iter):
        pbar = tqdm(range(n_iter), desc="ABC", unit="it")
        for _ in pbar:
            self.step()
            v0, v1 = self.gbest_val[0]
            pbar.set_postfix(viol=int(v0), obj=int(v1))
            time.sleep(0.01)  
        return self.gbest_val

    def optimize(self, n_iter):
        for _ in range(n_iter):
            self.step()
        return self.gbest_val

    def _print_setup(self):
        
        X = self.gbest_X
        O = self.gbest_O
        M, T = len(X), len(X[0])
        w_k = max(len("Product k"), len(str(M))) + 2
        w_c = max(len('-->'), 2) + 2
        total_width = w_k + T * (w_c + 1) + 2
        print("Setup sequence".center(total_width))
        def border():
            parts = ['+' + '-' * w_k] + ['+' + '-' * w_c for _ in range(T)]
            print(''.join(parts) + '+')
    
        border()
        merged_w = T * w_c + (T - 1)
        print(
            '|' + ' ' * w_k +
            '|' + 'Period t'.center(merged_w) +
            '|'
        )
        border()
        header = f"| {'Product k'.center(w_k-1)}"
        for t in range(1, T+1):
            header += f"| {str(t).center(w_c-1)}"
        print(header + "|")
        border()
    
        for m in range(M):
            row = f"| {str(m+1).ljust(w_k-1)}"
            for t in range(T):
                # arrow condition
                if 0 < t < T-1 \
                 and O[m][t-1] == 1 and X[m][t-1] == 1 \
                 and O[m][t+1] == 0 and X[m][t+1] == 1 \
                and  np.sum(O[:,t])==0 \
                and  np.sum(X[:,t])==0:
                    cell = '-->'
                else:
                    cell = ('x□' if O[m][t] and X[m][t]
                           else 'x'   if O[m][t]
                           else '□'   if X[m][t]
                           else '')
                row += f"| {cell.center(w_c-1)}"
            print(row + "|")
            border()
    def _print_prod_plan(self):

        M, T = self.gbest_X.shape[0], self.gbest_X.shape[1]
        w_k = max(len("Product k"), len(str(M))) + 2
        w_c = max(len('-->'), 2) + 2
        total_width = w_k + T * (w_c + 1) + 2
        plan = decode_prod_plan(self.gbest_X, self.gbest_Q, self.cfg.demand)
                # Production quantities
        print("Production quantities".center(total_width))
        def border():
            parts = ['+' + '-' * w_k] + ['+' + '-' * w_c for _ in range(T)]
            print(''.join(parts) + '+')
        border()
        print('|' + ' '*w_k + '|' + 'Period t'.center(T*w_c + T-1) + '|')
        border()
        header = "| " + "Product k".center(w_k-1)
        for t in range(1, T+1):
            header += f"| {str(t).center(w_c-1)}"
        print(header + "|")
        border()
        for m in range(M):
            row = "| " + str(m+1).ljust(w_k-1)
            for t in range(T):
                num = f"{plan[m][t]:.0f}"
                row += f"| {num.center(w_c-1)}"
            print(row + "|")
            border()

    def print(self):
        self._print_prod_plan()
        self._print_setup()



In [19]:
optimal = 0
results = []
bad_results = []
for _ in range(10):
    n = np.random.randint(0,1000)
    np.random.seed(n)
    # instantiate swarm
    start = time.perf_counter()
    abc = ABC(
        n_bees=100,
        config=CLSP1,
        limit=100,
        K_onlookers= 100,
        threshold1 = 0.15,
        threshold2 = 0.2)
    
    best_val = abc.optimize(5000)
    end = time.perf_counter()
    print(f"Elapsed time: {end - start:.4f} seconds")

    if best_val[0,1] <1711:
        #abc.print()
        optimal+=1
        
    end = time.perf_counter()

    if best_val[0, 0] == 0.0:
        results.append(best_val[0, 1])
    else:
        bad_results.append((best_val))

abc.print()

print(results)
print(f'number of optimal solutions {optimal}')
print('median', np.median(results))
print('infeasible', len(bad_results))


Elapsed time: 2.4530 seconds
Elapsed time: 2.2964 seconds
Elapsed time: 2.2930 seconds
Elapsed time: 2.2851 seconds
Elapsed time: 2.2720 seconds
Elapsed time: 2.2950 seconds
Elapsed time: 2.2773 seconds
Elapsed time: 2.3111 seconds
Elapsed time: 2.3034 seconds
Elapsed time: 2.3768 seconds
              Production quantities              
+-----------+-----+-----+-----+-----+-----+-----+
|           |              Period t             |
+-----------+-----+-----+-----+-----+-----+-----+
| Product k |  1  |  2  |  3  |  4  |  5  |  6  |
+-----------+-----+-----+-----+-----+-----+-----+
| 1         |  10 |  20 |  20 |  0  |  60 |  40 |
+-----------+-----+-----+-----+-----+-----+-----+
| 2         |  20 |  0  |  29 |  50 |  0  |  20 |
+-----------+-----+-----+-----+-----+-----+-----+
| 3         |  24 |  30 |  0  |  0  |  0  |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
                  Setup sequence                 
+-----------+-----+-----+-----+-----+-----+-----+
|         

In [11]:
abc.cfg.demand

array([[10., 20., 20.,  0., 35., 65.],
       [20.,  0.,  3., 18., 13., 65.],
       [ 2., 10., 12., 30.,  0.,  0.]])

In [58]:
abc.gbest_Q

array([[6.69924711e-01, 9.99418157e-01, 1.54919629e-04, 4.21656099e-04,
        9.99218799e-01, 7.66023134e-01],
       [1.11676451e-01, 8.68970416e-01, 2.71680715e-01, 7.32551799e-01,
        2.13802283e-03, 8.90912931e-01],
       [5.02851660e-02, 8.62249198e-01, 3.04105537e-03, 6.56849880e-02,
        1.74826328e-04, 5.44363224e-01],
       [3.31712090e-01, 8.92115160e-01, 3.40895386e-01, 6.48844056e-01,
        1.27362004e-01, 5.34898593e-03],
       [7.60187585e-01, 8.03497369e-01, 3.69319220e-01, 9.13449689e-01,
        4.52979929e-04, 7.91524617e-01]])

In [38]:
abc.print()

              Production quantities              
+-----------+-----+-----+-----+-----+-----+-----+
|           |              Period t             |
+-----------+-----+-----+-----+-----+-----+-----+
| Product k |  1  |  2  |  3  |  4  |  5  |  6  |
+-----------+-----+-----+-----+-----+-----+-----+
| 1         |  30 |  31 |  49 |  0  |  40 |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 2         |  0  |  0  |  31 |  0  |  69 |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 3         |  0  |  0  |  41 |  0  |  59 |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 4         |  0  |  0  |  30 |  0  |  0  |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 5         | 110 |  0  |  0  |  0  |  0  |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
                  Setup sequence                 
+-----------+-----+-----+-----+-----+-----+-----+
|           |              Period t             |
+-----------+-----+-----+-----+-----+-----+-----+


In [285]:
abc.gbest_Q

array([[2.92862676e-01, 5.69509123e-01, 1.33223559e-04, 9.99871063e-01,
        3.99421527e-01, 7.04757201e-01],
       [2.94511119e-01, 1.31623727e-04, 7.87426693e-03, 9.99257694e-01,
        8.79772925e-01, 7.72383245e-02],
       [9.96544282e-01, 1.20171960e-04, 1.75248448e-03, 4.42308651e-03,
        9.88467484e-01, 9.99869335e-01]])

In [276]:
abc.optimize(100)

array([[  23., 1650.]])

In [23]:
abc.print()


              Production quantities              
+-----------+-----+-----+-----+-----+-----+-----+
|           |              Period t             |
+-----------+-----+-----+-----+-----+-----+-----+
| Product k |  1  |  2  |  3  |  4  |  5  |  6  |
+-----------+-----+-----+-----+-----+-----+-----+
| 1         |  30 |  30 |  50 |  0  |  40 |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 2         |  0  |  0  |  30 |  0  |  70 |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 3         |  40 |  0  |  0  |  0  |  60 |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 4         |  0  |  0  |  30 |  0  |  0  |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
| 5         |  0  |  0  |  60 |  50 |  0  |  0  |
+-----------+-----+-----+-----+-----+-----+-----+
                  Setup sequence                 
+-----------+-----+-----+-----+-----+-----+-----+
|           |              Period t             |
+-----------+-----+-----+-----+-----+-----+-----+


In [10]:
setups

array([[1, 1, 1, 0, 0, 0],
       [0, 0, 1, 1, 1, 0],
       [1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0]], dtype=int8)

In [13]:
@njit
def decode_and_evaluate(X, Q, O, demand, setup_costs, production_costs,
                        production_times, setup_times, capacities, inventory_costs, idle_carry=2):

    T = X.shape[1]
    M = X.shape[0]

    prod_quant = np.zeros((M, T))

    # ------------------------------------------------------------------------------------------------------------------ # 
    # 1. Decode the production plan
    
    for i in range(M):
        for t in range(T):
            # skip non production periods
            if X[i, t] == 0:
                continue
            else:
                
                # Find next production period t2
                t2 = T - 1
                for j in range(t+1, T):
                    if X[i, j] == 1:
                        t2 = j
                        break
                # ---------------------------------------------------------------------------------------------------------- #
                # t = 1
                if t == 0:
                    # Case 1: no production is planned in the future
                    if (t2 == T - 1) and (X[i, t2] == 0):
                        s = 0.0
                        for j in range(t, T):
                            s += demand[i, j]
                        
                        prod_quant[i, t] = s
                        
                    # Case 2:  production is planned in the future
                    else:
                        # find production period after the next production period
                        t3 = T - 1
                        for j in range(t2+1, T):
                            if X[i, j] == 1:
                                t3 = j
                                break

                        end = t3 + 1 if (X[i, t3] == 0 or t3 == t2) else t3
   
                        s1 = 0.0
                        for j in range(t2, end):
                            s1 += demand[i, j]
                            
                            
                        s2 = 0.0                       
                        for j in range(t, t2):
                            s2 += demand[i, j]
                            
                        prod_quant[i, t] = rounded(s2 + Q[i, t2] * s1)
                        
                # ---------------------------------------------------------------------------------------------------------- #
                # t = T
                elif t == T - 1:
                    
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if X[i, j] == 1:
                            t1 = j
                            break
                            
                    # Case 3: no prior production before the last period 
                    if (t1==0) and (X[i,t1]==0):
                        
                        prod_quant[i, t] = demand[i,t]
                        
                    # Case 4: prior production 
                    else:
                        prod_quant[i, t] = rounded((1-Q[i, t]) * demand[i, t])
                          
                # ----------------------------------------------------------------------------------------------------------#
                # t in [2,T-1]
                else:
                    # Look backwards for a prior production period t1
                    t1 = 0
                    for j in range(t-1, -1, -1):
                        if X[i, j] == 1:
                            t1 = j
                            break

                    # Case 5: no production before AND after t, cover demand for t to T ; dont cover backorders
                    if (X[i,t1]==0) and (X[i,t2]==0):
                        s = 0.0
                        for j in range(t, T):
                            s += demand[i, j]
                        prod_quant[i, t] = s
                        
                        
                    # Case 6: production before but not after t              
                    elif (X[i,t1]==1) and (X[i,t2]==0):
                        s = 0.0
                        for j in range(t, t2+1):
                            s += demand[i, j]
                        
                        prod_quant[i, t] = rounded((1 - Q[i, t]) * s)

                    # Case 7: production after but not before t 
                    elif (X[i,t1]==0) and (X[i,t2]==1):
                        t3 = T - 1
                        for j in range(t2+1, T):
                            if X[i, j] == 1:
                                t3 = j
                                break
                                
                        end = t3 + 1 if (X[i, t3] == 0 or t3 == t2) else t3
                        s1 = 0.0
                        for j in range(t2, end):
                            s1 += demand[i, j]
                            
                        s2 = 0.0
                        for j in range(t, t2):
                            s2 += demand[i, j]

                        prod_quant[i, t] = rounded(s2 + Q[i, t2] * s1)
                        
                        
                    # Case 8: production after and before t  
                    else:
                        t3 = T - 1
                        for j in range(t2+1, T):
                            if X[i, j] == 1:
                                t3 = j
                                break
                        end = t3 + 1 if (X[i, t3] == 0 or t3 == t2) else t3
   
                        s1 = 0.0
                        for j in range(t2, end):
                            s1 += demand[i, j]
   
                        s2 = 0.0
                        for j in range(t, t2):
                            s2 += demand[i, j]

                        prod_quant[i, t] = rounded((1-Q[i, t]) * s2 + Q[i, t2] * s1)
                        

            # Clamp negatives
            if prod_quant[i ,t] < 0.0:
                prod_quant[i, t] = 0.0

    # ------------------------------------------------------------------------------------------------------------------ # 
    # 2. Check the setup sequence for feasibility

    violations = 0.0
    # no product is setup initially, use negative values
    last_setup = np.full(M, -10_000_000, np.int64)
    

    for t in range(T):
        #print('last setup', last_setup)

        # 1) production without explicit setup
        carried = -1
        max_last = -10_000_000
        # pick carried candidate: most recent within idle window
        for m in range(M):
            if X[m, t] == 1 and O[m, t] == 0:
                
                if t - last_setup[m] <= idle_carry:
                    if last_setup[m] > max_last:
                        max_last = last_setup[m]
                        carried = m

        # 2) apply carry
        if carried >= 0:
            #print(f"Carry-over setup for product {carried} at period {t}")
            last_setup[carried] = t
            # invalidate all other prior setup states when we carry over one state
            for m in range(M):
                if m != carried:
                    last_setup[m] = -10_000_000

        # 3) explicit setups
        did_setup = False
        for m in range(M):
            if O[m, t] == 1:
 
                #print(f"Setup cost for product {k} at period {t}: +{setup_costs[k]}")
                last_setup[m] = t
                did_setup = True

        # 4) reconfiguration: if any explicit setup, lose carry for others
        if did_setup:
            for m in range(M):
                if O[m, t] == 0:
                    last_setup[m] = -10_000_000


        # 5) penalties for others
        penalty_occurred = False
        for m in range(M):
            if X[m, t] == 1 and O[m, t] == 0 and m != carried:
                violations +=1
                penalty_occurred = True

        # 6) after any penalty, invalidate all prior setup states
        if penalty_occurred:
            for m in range(M):
                last_setup[m] = -10_000_000
            carried = -1
        #print(last_setup)
                
   
    
    # ------------------------------------------------------------------------------------------------------------------ # 
    # 3. Determine total costs, backorders and overtime

                
    net_inv = np.zeros((M, T))
    inv = np.zeros((M, T))
    prod_time = np.zeros(T)
    setup_time = np.zeros(T)
    total_cost = 0.0

    # Inventory and violations
    for i in range(M):
        for t in range(T):
            if t == 0:
                net_inv[i, t] = prod_quant[i, t] - demand[i, t]
            else:
                net_inv[i, t] = net_inv[i, t - 1] + prod_quant[i, t] - demand[i, t]

            if net_inv[i, t] >= 0:
                inv[i, t] = net_inv[i, t]
            else:
                violations += -net_inv[i, t]  # backorder penalty
            if X[i,t]== 1 and prod_quant[i,t]==0:
                violations +=1
                

    # Cost and time
    for t in range(T):
        for i in range(M):
            total_cost += production_costs[i] * prod_quant[i, t]
            total_cost += setup_costs[i] * O[i, t]
            total_cost += inventory_costs[i] * inv[i, t]
            prod_time[t] += prod_quant[i, t] * production_times[i]
            setup_time[t] += O[i, t] * setup_times[i]

        ot = prod_time[t] + setup_time[t] - capacities[t]
        if ot > 0:
            violations += ot

    return np.array([violations, total_cost], dtype=np.float64)

In [4]:
X = abc.X
Q = abc.Q
for i in range(100):
    plan1 = decode_prod_plan(X[i], Q[i], abc.cfg.demand)
    X1, Q1 = encode_prod_plan(plan1, abc.cfg.demand)
    plan2 = decode_prod_plan(X1, Q1, abc.cfg.demand)
    if np.any(plan1 != plan2):
        print(i, plan1 - plan2)
        print('#--------------#')

34 [[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
#--------------#
